In [1]:
import requests
from datetime import datetime
import json
import time
import pandas as pd
from time import mktime

In [2]:
class PolgonData(object):

    def __init__(self):
        self.params=params = (('apiKey', 'M_PKVL_rqHZI7VM9ZYO_hwPiConz5rIklx893F'),)

    def PolygonLastTrades(self,symbol):
        # Make use of Tickers
        requesturl='https://api.polygon.io/v1/last/stocks/'+symbol
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

    def PolygonHistoricTrades(self, date=None, symbol=None,startTS=None,endTS=None,limitresult=10):
        if startTS:
            # For Getting Paginated Request
            requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestamp='+startTS+'&timestampLimit='+endTS+'&limit='+limitresult
            print("Paginated Request For = " + symbol)
        else:
            requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestampLimit='+endTS+'&limit='+limitresult
            print("First Request For = " + symbol)
        print(requesturl)
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)
    
    def PolygonAggregdateData(self):
        # Make use of Tickers, Date and Limit
        requesturl='https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2020-02-14/2020-02-15'
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

In [3]:
from datetime import datetime

class DateTimeManipulation(object):
    
    def __init__(self,date=None):
        self.date=date

    # Returns timestamp with milliseconds
    def unix_time_millis(self,dt):
        epoch = datetime.utcfromtimestamp(0)
        tsDate=(dt - epoch).total_seconds() * 1000.0
        tsDate=str(int(tsDate))+'000000'
        return tsDate

    def stringTimeToDatetime(self,date=None,time=None):
        marketOpenTSStr = date +' '+ time
        return datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S')
    
    def convertStringDateToTS(self,starttime='9:30:00',endtime='4:00:00'):
        marketOpenTSStr = self.date +' '+ starttime
        marketCloseTSStr = self.date +' ' + endtime
        
        marketTimeStamps={}
        marketTimeStamps['marketOpenTS']=self.unix_time_millis(datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S'))
        marketTimeStamps['marketCloseTS']=self.unix_time_millis(datetime.strptime(marketCloseTSStr,'%Y-%m-%d %H:%M:%S'))
        return marketTimeStamps
    
    def getHumanTime(self,ts,getMilliSecondsAlso=False):
        try:
            s, ms = divmod(ts, 1000000000)
            if getMilliSecondsAlso:
                return datetime(*time.gmtime(s)[:6]),ms
            else:
                return datetime(*time.gmtime(s)[:6])
            #print('{}.{:03d}'.format(time.strftime('%Y-%m-%d %H:%M:%S',  time.gmtime(s)), ms))
        except AttributeError:
            print("Attribute Error Occured")
            print(ts)
            print(s)
            print(ms)
            

# Process the ticker data

XLK: Technology Select Sector SPDR Fund <br>
Inception Date: 1998-12-16 <br>
Fund Holdings as of: 2020-02-21 <br>
"Total Assets Under Management (in thousands):	28583351" <br>
Shares Outstanding: 287356000 <br>
Expense Ratio: 0.13% <br>
Tracks This Index: Technology Select Sector Index <br>
ETFdb.com Category: Technology Equities <br>
Issuer: State Street SPDR <br>
Structure: ETF <br>

In [13]:
date='2020-02-21'
starttime='9:30:00'
endtime='17:00:00'

endtimeLoop='16:00:00'
extractDataTillTime = DateTimeManipulation().stringTimeToDatetime(date=date,time=endtimeLoop)

# Taking in ETF List
data=pd.read_csv("XLK-holdings20202302.csv")
symbols=list(data['Symbol'].values)+['XLK']

# Create an object of date when we need and time between which we need data
datetimeObj=DateTimeManipulation(date)
marketTimeStamps=datetimeObj.convertStringDateToTS(starttime=starttime,endtime=endtime)

tickHistData={}
for symbol in symbols:
    # First Request
    data=PolgonData().PolygonHistoricTrades(date=date,symbol=symbol,endTS=marketTimeStamps['marketCloseTS'],limitresult=str(50000))
    
    # Last timestamp from data received
    lastUnixTimeStamp = data['results'][-1]['t']
    # Covert UNIX timestamp to human timestamp
    lastHumanTimeStamp = datetimeObj.getHumanTime(lastUnixTimeStamp)
    # Get timestamp for date +  '18:00:00' hrs - Make use of pagination
    
    # Paginated Request
    while lastHumanTimeStamp < extractDataTillTime:
        print(datetimeObj.getHumanTime(data['results'][-1]['t']))
        data2=PolgonData().PolygonHistoricTrades(date=date,symbol=symbol,startTS=str(lastUnixTimeStamp),endTS=marketTimeStamps['marketCloseTS'],limitresult=str(50000))
        # Last timestamp from data received
        lastUnixTimeStamp = data2['results'][-1]['t']
        # Covert UNIX timestamp to human timestamp
        lastHumanTimeStamp = datetimeObj.getHumanTime(lastUnixTimeStamp)
        # Get timestamp for date +  '18:00:00' hrs - Make use of pagination
        data['results']=data['results'] + data2['results']
        
    tickHistData[symbol]=data
    print(lastHumanTimeStamp)

First Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-21 14:46:37
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582296397554225623&timestampLimit=1582304400000000000&limit=50000
2020-02-21 14:58:15
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582297095661536527&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:16:22
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582298182134773058&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:41:34
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-02-21?timestamp=1582299694220837481&timestampLimit=1582304400000000000&limit=50000
2020-02-21 16:11:06
First Request For = AAPL
https://api.polygon.io/v2/ticks/stocks/nbbo/AAPL/2020-02-21?timestampLimit=1582

2020-02-21 18:32:20
First Request For = ADSK
https://api.polygon.io/v2/ticks/stocks/nbbo/ADSK/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-21 20:53:29
First Request For = CTSH
https://api.polygon.io/v2/ticks/stocks/nbbo/CTSH/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-21 20:00:56
First Request For = HPQ
https://api.polygon.io/v2/ticks/stocks/nbbo/HPQ/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:21:08
Paginated Request For = HPQ
https://api.polygon.io/v2/ticks/stocks/nbbo/HPQ/2020-02-21?timestamp=1582298468853459198&timestampLimit=1582304400000000000&limit=50000
2020-02-21 16:46:34
First Request For = MSI
https://api.polygon.io/v2/ticks/stocks/nbbo/MSI/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-22 01:00:00
First Request For = TEL
https://api.polygon.io/v2/ticks/stocks/nbbo/TEL/2020-02-21?timestampLimit=1582304400000000000&limit=50000
2020-02-21 22:15:21
First Request For = APH
https://api.polygo

2020-02-21 14:55:15
Paginated Request For = XLK
https://api.polygon.io/v2/ticks/stocks/nbbo/XLK/2020-02-21?timestamp=1582296915676768082&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:05:38
Paginated Request For = XLK
https://api.polygon.io/v2/ticks/stocks/nbbo/XLK/2020-02-21?timestamp=1582297538354817349&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:14:23
Paginated Request For = XLK
https://api.polygon.io/v2/ticks/stocks/nbbo/XLK/2020-02-21?timestamp=1582298063283065768&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:23:15
Paginated Request For = XLK
https://api.polygon.io/v2/ticks/stocks/nbbo/XLK/2020-02-21?timestamp=1582298595117764540&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:37:05
Paginated Request For = XLK
https://api.polygon.io/v2/ticks/stocks/nbbo/XLK/2020-02-21?timestamp=1582299425140907194&timestampLimit=1582304400000000000&limit=50000
2020-02-21 15:47:33
Paginated Request For = XLK
https://api.polygon.io/v2/ticks/

# Meaning of Each Columns

In [14]:
tickHistData['AAPL']['map']

{'i': {'name': 'indicators', 'type': '[]int'},
 's': {'name': 'bid_size', 'type': 'int'},
 'x': {'name': 'bid_exchange', 'type': 'int'},
 'P': {'name': 'ask_price', 'type': 'float64'},
 'S': {'name': 'ask_size', 'type': 'int'},
 't': {'name': 'sip_timestamp', 'type': 'int64'},
 'f': {'name': 'trf_timestamp', 'type': 'int64'},
 'c': {'name': 'conditions', 'type': '[]int'},
 'z': {'name': 'tape', 'type': 'int'},
 'X': {'name': 'ask_exchange', 'type': 'int'},
 'y': {'name': 'participant_timestamp', 'type': 'int64'},
 'q': {'name': 'sequence_number', 'type': 'int'},
 'p': {'name': 'bid_price', 'type': 'float64'}}

In [237]:
holdings=pd.read_csv("XLK-holdings20202302.csv")
holdings['Weighting']=holdings['Weighting'].apply(lambda x:x.replace('%',''))
holdings['Weighting']=holdings['Weighting'].astype(float)
holdings['Weighting']=holdings['Weighting']/100
weights=dict(zip(holdings.Symbol,holdings.Weighting))

In [15]:
finalDF=[]
appendData=[]
for key,value in tickHistData.items():
    df=pd.DataFrame.from_dict(value['results'])
    df['Symbol']=key
    appendData.append(df)

finalDF= pd.concat(appendData)tart

In [16]:
# Delete conditions
del finalDF['c']
# Delete sequence number
del finalDF['q']
# Delete indicators
del finalDF['i']

In [17]:
# Clean the DataFrame
finalDF['t']=finalDF['t'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
finalDF['y']=finalDF['y'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
finalDF['Spread']=finalDF['P']-finalDF['p']
finalDF['MidPrice']=(finalDF['P']+finalDF['p'])/2

In [257]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [29]:
#finalDF.to_csv("CombinedDF.csv")
#finalDF=pd.read_csv("CombinedDF.csv")
#finalDF.t=pd.to_datetime(finalDF['t'])

finalDF= finalDF[finalDF['Symbol']!='CASH']
dfGroupedByObject=finalDF.groupby([finalDF['t'].dt.hour,finalDF['t'].dt.minute,finalDF['Symbol']])['MidPrice'].mean()

In [248]:
etfspreadgroupby=finalDF[finalDF['Symbol']=='XLK']
etfspread=etfspreadgroupby.groupby([etfspreadgroupby['t'].dt.hour,etfspreadgroupby['t'].dt.minute])['Spread'].mean()

In [49]:
df=dfGroupedByObject.unstack(level=2)

In [51]:
df=df.fillna(method='ffill')

In [224]:
for name,group in df.groupby(level=0):
    if name==15:
        break

In [225]:
etfprice=group['XLK']
del group['XLK']

In [226]:
group

Symbol        AAPL         ACN        ADBE         ADI         ADP         ADS        ADSK        AKAM       AMAT        AMD        ANET        ANSS         APH        AVGO          BR       CDNS         CDW         CRM       CSCO       CTSH        CTXS        DXC        FFIV         FIS        FISV       FLIR         FLT        FTNT        GLW         GPN        HPE        HPQ         IBM       INTC        INTU        IPGP          IT        JKHY       JNPR       KEYS        KLAC        LDOS        LRCX          MA        MCHP        MSFT         MSI         MU       MXIM       NLOK         NOW       NTAP        NVDA       ORCL        PAYC       PAYX        PYPL       QCOM       QRVO        SNPS        STX        SWKS        TEL         TXN           V        VRSN        WDC         WU       XLNX        XRX        ZBRA
t  t                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
15 0    317.591773  211.493689  373.790118  121.861027  179.498233  101.708926  203.272741  100.099853  64.482992  55.247795  222.297857  288.837742  102.890038  303.192340  120.026458  75.042156  135.858000  188.566943  46.468547  69.078207  118.505789  29.498048  131.119111  155.511510  120.445953  57.807500  310.627468  115.397459  27.869391  204.401469  14.268845  22.669336  149.145309  64.689177  297.515981  137.118333  154.104912  172.423000  24.404181  96.238355  162.425875  116.411282  320.184021  339.268966  105.823049  180.311559  184.579323  57.709561  61.348862  20.779434  347.296875  52.636267  299.666057  54.833348  303.807921  89.416154  120.148759  87.088825  99.661596  150.864788  54.346542  114.526125  91.684180  128.679516  208.842232  206.662200  67.256389  25.970812  87.568681  36.425547  237.655070
   1    317.398926  211.496016  373.587888  121.702901  179.414848  101.385067  203.179737  100.046538  64.405469  55.254896  222.273110  288.695056  102.730759  302.984554  120.028667  74.994647  135.791582  188.740066  46.437299  69.062189  118.595341  29.504656  131.087941  155.445071  120.402097  57.779062  310.470000  115.392000  27.844228  204.464362  14.257483  22.650429  148.931935  64.621483  297.242247  136.683141  153.906711  172.366727  24.368844  96.115848  162.306886  116.375000  319.630942  339.121927  105.626084  180.371179  184.468511  57.635537  61.272059  20.779434  346.933114  52.617713  299.798202  54.784075  303.740231  89.399767  120.068760  87.015684  99.400789  150.649422  54.284533  114.322214  91.551172  128.541132  208.744446  206.555000  67.132729  25.908515  87.419587  36.429384  237.622849
   2    317.133386  211.424697  372.992689  121.504747  179.327048  101.237759  202.829836   99.978144  64.352732  55.164702  221.757215  288.188265  102.606667  303.153131  119.900462  74.870205  135.617436  188.441472  46.417441  69.068833  118.578447  29.445810  131.014778  155.392540  120.275689  57.772143  310.201809  115.197615  27.829170  204.444880  14.254129  22.630123  148.921103  64.600838  297.157731  136.537437  153.829200  172.168132  24.346118  95.963483  162.243889  116.136139  319.375270  338.777804  105.504430  180.310414  184.274444  57.478107  61.234431  20.767394  346.053779  52.611049  299.390228  54.753140  302.853211  89.397544  119.806924 

In [243]:
group=group.pct_change()*100
group=group.dropna()

In [244]:
etfpricechange=etfprice.pct_change().dropna()*100
etfpricechange=etfpricechange.unstack(level=1)

netassetvaluereturn=group.assign(**weights).mul(group).sum(axis=1)
netassetvaluereturn=netassetvaluereturn.unstack(level=1)

In [283]:
ds=pd.concat([etfprice.unstack(level=1),etfpricechange,netassetvaluereturn],axis=0).T
ds.columns=['ETF Price','ETF Change Price %','Net Asset Value Change%']

In [284]:
ds['Arbitrage in $']=(ds['ETF Change Price %'] - ds['Net Asset Value Change%'])*ds['ETF Price']/100
ds['ETF Trading Spread in $']=etfspread.unstack(level=1).loc[15]

In [290]:
ds

,ETF Price,ETF Change Price %,Net Asset Value Change%,Arbitrage in $,ETF Trading Spread in $
t,,,,,
0,100.119986,NaN,NaN,NaN,0.010625
1,100.076029,-0.043904,-0.041379,-0.002527,0.010319
2,100.000213,-0.075759,-0.080033,0.004274,0.010427
3,99.969048,-0.031165,-0.027950,-0.003214,0.010000
4,99.954277,-0.014775,-0.025331,0.010552,0.009974
5,99.952841,-0.001437,0.006915,-0.008348,0.010103
6,100.012761,0.059949,0.059923,0.000026,0.010067
7,100.067342,0.054573,0.053675,0.000899,0.010227
8,100.047273,-0.020055,-0.024812,0.004759,0.010285


In [288]:
#p=finalDF[finalDF['Symbol']=='XLK']
#p[p['t'].dt.hour==16]